# Introdução

Este notebook tem o intuito de relacionar as bases 1 e 2 conforme descritos abaixo:
    
1. "Emission_Country_Year" - Base principal do projeto, pois contén os dados quantitativos relacionados a emissão de gases poluentes por País e Ano, esta base registra dados de 1850 a 2019 de 236 países.

2. "Data-gdp-per-capita" - Base auxiliar, contém informações sobre o PIB de cada País por ano.

As relações das bases se darão pelas colunas Base 1, Coluna "country" e Base 2, coluna "Entity" que são comuns em ambas as bases.

Os dados foram limitados para dar inicio no ano de 1850, por conta da nossa base principal iniciar neste ano.

Contextualização: Utilizaremos está base para dar mais detalhes de como a Emissão de cO2 influenciou na economia de cada País. 

# Configurações e importações de bibliotecas

In [1]:
!pip install google-cloud-storage
!pip install pandas

In [2]:
# Importar a biblioteca do 
import pandas as pd

# Importar a biblioteca do Google Cloud Storage para upload do arquivo .csv salvo
from google.cloud import storage

# Criação da Dimensão País 

In [20]:
# Ler a base dimensão País do Github
Pais = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv', error_bad_lines=False)

# Listar todas as colunas para visualizar quais serão utilizadas
Pais.columns

Index(['iso_code', 'country', 'year', 'co2', 'co2_growth_prct',
       'co2_growth_abs', 'consumption_co2', 'trade_co2', 'trade_co2_share',
       'co2_per_capita', 'consumption_co2_per_capita', 'share_global_co2',
       'cumulative_co2', 'share_global_cumulative_co2', 'co2_per_gdp',
       'consumption_co2_per_gdp', 'co2_per_unit_energy', 'cement_co2',
       'coal_co2', 'flaring_co2', 'gas_co2', 'oil_co2', 'other_industry_co2',
       'cement_co2_per_capita', 'coal_co2_per_capita',
       'flaring_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'other_co2_per_capita', 'share_global_coal_co2', 'share_global_oil_co2',
       'share_global_gas_co2', 'share_global_flaring_co2',
       'share_global_cement_co2', 'cumulative_coal_co2', 'cumulative_oil_co2',
       'cumulative_gas_co2', 'cumulative_flaring_co2', 'cumulative_cement_co2',
       'share_global_cumulative_coal_co2', 'share_global_cumulative_oil_co2',
       'share_global_cumulative_gas_co2',
       'share_g

In [21]:
# Selecionar as colunas que serão utilizadas
Pais = Pais.filter(items=['iso_code', 'country'])

# Alterar todos os valores nulos em iso_code
Pais['iso_code'].fillna('Preencher',inplace=True)
Pais

,iso_code,country
0,AFG,Afghanistan
1,AFG,Afghanistan
2,AFG,Afghanistan
3,AFG,Afghanistan
4,AFG,Afghanistan
...,...,...
23703,ZWE,Zimbabwe
23704,ZWE,Zimbabwe
23705,ZWE,Zimbabwe
23706,ZWE,Zimbabwe


In [22]:
#Agrupar 'iso_code' selecionando os dados coluna 'country' - índice reiniciado para 'iso_code' e transformado em coluna
Pais = Pais.groupby('country')['iso_code'].apply(lambda Pais: Pais.reset_index(drop=True)).unstack(0)

#Dropar todos os valores nulos
Pais.dropna(inplace=True)

# Transpor DataFrame
Pais = Pais.transpose()

# Reiniciar índice
Pais.reset_index(inplace=True)

# Alterar nome da coluna para 'country'
Pais.columns = [['country', 'iso_code']]

#exibição do dataframe
Pais.head()

,country,iso_code
0,Afghanistan,AFG
1,Africa,Preencher
2,Albania,ALB
3,Algeria,DZA
4,Andorra,AND


# Tratamento de PIB por País

In [37]:
# Ler a base do PIB do diretorio do GitHub
PibPais = pd.read_csv('https://raw.githubusercontent.com/ViniGome/Projeto-Integrado/main/maddison-data-gdp-per-capita-in-2011us-single-benchmark.csv', error_bad_lines=False)

# Listar todas as colunas para visualizar quais serão utilizadas
PibPais.columns

Index(['Entity', 'Code', 'Year',
       'Real GDP per capita in 2011US$, 2011 benchmark (Maddison Project Database (2018))'],
      dtype='object')

In [38]:
# Selecionar as colunas que serão utilizadas
PibPais = PibPais.filter(items=['Year', 'Entity', 'Real GDP per capita in 2011US$, 2011 benchmark (Maddison Project Database (2018))'])

In [39]:
# Dropando dados abaixo do ano de 1850.
indexNames = PibPais[(PibPais['Year'] < 1850 )].index
PibPais.drop(indexNames, inplace=True)
PibPais.reset_index(inplace = True)
PibPais.drop(['index'],axis=1, inplace=True)
PibPais

,Year,Entity,"Real GDP per capita in 2011US$, 2011 benchmark (Maddison Project Database (2018))"
0,1950,Afghanistan,2392
1,1951,Afghanistan,2422
2,1952,Afghanistan,2462
3,1953,Afghanistan,2568
4,1954,Afghanistan,2576
...,...,...,...
14643,2012,Zimbabwe,1604
14644,2013,Zimbabwe,1604
14645,2014,Zimbabwe,1594
14646,2015,Zimbabwe,1560


In [40]:
# Converter coluna dt para número inteiro
PibPais = PibPais.astype({'Year': int})

# Alterar nomes das colunas
PibPais = PibPais.rename(columns={'Entity': 'Country', 'Real GDP per capita in 2011US$, 2011 benchmark (Maddison Project Database (2018))': 'PIB'})

In [41]:
#importação de pacote
from google.cloud import storage
# setando as credeciais com o arquivo json que contem a chave
client = storage.Client.from_service_account_json(json_credentials_path='/home/greyce_cos_sil/projetoteste-299919-abc73be2d5b5.json')
# setando o bucket onde deseja-se salvar os arquivos
bucket = client.get_bucket('jupyter_note')
#salvando os dados do dataframe Kyoto_Agreement_Countries em um arquivo csv
PibPais.to_csv("PIB_Country_Year.csv")
#Nome do arquivo que será salvo no bucket
PIB_Country_Year_bucket = bucket.blob('PIB_Country_Year.csv')
# Name of the object in local file system
PIB_Country_Year_bucket.upload_from_filename('/home/jupyter/PIB_Country_Year.csv')

In [42]:
# Calcular as médias do PIB por Ano e País.
MediaPibPais = PibPais.groupby(['Year', 'Country']).mean()

# Visualizar o novo dataframe
MediaPibPais.head()

PIB
Year Country        
1850 Argentina  2198
     Australia  3589
     Austria    2861
     Belgium    3029
     Brazil     1123

In [43]:
#salvando os dados do dataframe Kyoto_Agreement_Countries em um arquivo csv
MediaPibPais.to_csv("Media_PIB_Country_Year.csv")
#Nome do arquivo que será salvo no bucket
MediaPibPais_Country_Year_bucket = bucket.blob('Media_PIB_Country_Year.csv')
# Name of the object in local file system
MediaPibPais_Country_Year_bucket.upload_from_filename('/home/jupyter/Media_PIB_Country_Year.csv')

Autor do notebook >>>>>>>>>>>>>>>>>>> Vinicius